In [47]:
import pandas as pd
from lbmpy_tests.benchmark.evaluation import *
import seaborn as sns
from bokeh.io import output_notebook, save, show
from bokeh.layouts import gridplot
from bokeh.resources import Resources
output_notebook()

Loading BokehJS ...

In [50]:
df = get_categorical({'study_name': 'vectorization_flags'})
df = df.query('openmp==4 and stencil=="D3Q19" and method=="mrt_raw"')

In [57]:
out = df.sort_values(['fixed', 'method', 'split'])
out = out.reset_index()
del out['pk']
out

,compiler_flags,method,openmp,split,instruction_set,stencil,mlups_max,mlups_median,fixed
0,-march=native -mavx512vl,mrt_raw,4,no-split,avx512,D3Q19,105.689198,105.668004,generic
1,-march=native -mavx512vl -mno-fma,mrt_raw,4,no-split,avx512,D3Q19,105.549041,104.331998,generic
2,-mavx2,mrt_raw,4,no-split,avx,D3Q19,9.313466,9.303262,generic
3,-mavx2 -mno-fma,mrt_raw,4,no-split,avx,D3Q19,9.260901,9.218732,generic
4,-march=native -mavx512vl,mrt_raw,4,split,avx512,D3Q19,103.796639,103.735438,generic
5,-march=native -mavx512vl -mno-fma,mrt_raw,4,split,avx512,D3Q19,104.085830,102.983059,generic
6,-mavx2,mrt_raw,4,split,avx,D3Q19,79.212026,78.240678,generic
7,-mavx2 -mno-fma,mrt_raw,4,split,avx,D3Q19,77.784685,77.739951,generic
8,-march=native -mavx512vl,mrt_raw,4,no-split,avx512,D3Q19,105.555956,105.546385,all fixed
9,-march=native -mavx512vl -mno-fma,mrt_raw,4,no-split,avx512,D3Q19,106.581461,105.378463,all fixed


# Evaluation of optimization study

study compares different optimization options for two domain sizes of a 2D domain $1024^2$ and a 3D domain $128^3$ on a single socket of the benchmark machine using AVX and AVX512 parallelization.

In [ ]:
opt_study_3d = get_categorical({'study_name': 'optimization_options',
                                'domain_size':[128, 128, 128]}).query('stable==True')
opt_study_2d = get_categorical({'study_name': 'optimization_options',
                                'domain_size':[1024, 1024]})#.query('stable==True')

In [ ]:
#opt_study_3d.query("method=='srt-cumulant'")['stencil']

### All D3Q27 single core results

In [ ]:
df = opt_study_3d.query("openmp == 1 and stencil=='D3Q19'")
#df = opt_study_2d.query("openmp == 1 and method=='mrt_raw'")

ps = (980, 150)
res_col = 'mlups_max'
p1, src = bokeh_scatter_plot(df, 'vec', res_col, color_column='split', plot_size=ps)
p2, src = bokeh_scatter_plot(df, 'fixed', res_col, color_column='cse', source=src, plot_size=ps)
p3, src = bokeh_scatter_plot(df, 'cse', res_col, color_column='field_layout', source=src, plot_size=ps)
p4, src = bokeh_scatter_plot(df, 'method', res_col, color_column='cse', source=src, plot_size=ps)

p1.legend.label_text_font_size="6pt"
p1.legend.padding=1
p1.legend.margin=1
pl = gridplot([[p1], [p2], [p3], [p4]])

save(pl, filename="plot.html", title="lbmpy Benchmark", resources=Resources('inline'))
show(pl)

## Speedup plots

In [ ]:
from bokeh.models import Span

df = opt_study_3d.query("openmp == 4 and stencil=='D3Q27'")
speedup = speedup_table_categorical(df, 'fixed', ['generic', "loops only"])

ps = (450, 300)
p1, src = bokeh_scatter_plot(speedup, 'vec', 'speedup', color_column='split', 
                             plot_size=ps, log=True)
p2, src = bokeh_scatter_plot(speedup, 'method', 'speedup', color_column='cse', 
                             plot_size=ps, source=src, log=True)

# Vertical line at 1
p1.renderers.extend([Span(location=1, dimension='height', line_color='black', line_width=1)])

pl = gridplot([[p1, p2]])
show(pl)

### Best runs for each method

In [ ]:
res_col = 'mlups_median'
df = opt_study_3d.query("openmp == 4 and stencil=='D3Q19'")
#df = opt_study_2d.query("openmp == 1")

r = df.groupby(['method'])[[res_col]].max().reset_index()
r = r.sort_values(by=res_col, ascending=False)
sns.barplot(y = 'method', x=res_col, data=r)
r

## Seaborn - tests

In [ ]:
df = opt_study_3d.query("openmp == 4 and stencil=='D3Q27'")
speedup = speedup_table_categorical(df, 'fixed', ['generic', "loops only"])

ax = sns.stripplot(y='speedup', x='cse', jitter=True, hue='split', data=speedup)
ax = sns.violinplot(y='speedup', x='cse', data=speedup, inner=None, color=".8")

In [ ]:
import matplotlib.pyplot as plt
data = opt_study_3d.query("stencil=='D3Q27'")
fig = plt.gcf()
ax = sns.stripplot(y='mlups_max', x='openmp', jitter=True, hue='split', data=data)
ax = sns.violinplot(y='mlups_max', x='openmp', data=data, inner=None, color=".8")

In [ ]:
sns.stripplot(y='mlups_max', x='openmp', jitter=True, hue='cse', data=data)
sns.violinplot(y='mlups_max', x='openmp', data=data, inner=None, color=".8")

In [ ]:
sns.violinplot(data=speedup.query("speedup<1.5")['speedup'], orient='h')